# A Bandit Approach to Music Recommendation

## Background

### Music Recommendation

Music recommendation methods can be classified into four categories:
* **Collaborative Filtering**

Collaborative filtering is a method of making automatic predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). [wikipedia](https://en.wikipedia.org/wiki/Collaborative_filtering)

* **Content-based Methods**

Content-based methods recommends songs which have similiar audio content features (such as timbral, temporal, and tonal) to the songs liked by the user.

* **Context-Based Methods**

Context-based recommender systems propose recommended songs to match various aspects of the user such as mood, pyschological state, environmetn the user is in, etc ...

* **Hybrid Methods**

Hybrid methods usually makes use of all of the methods mentioned above.

Although collaborative filtering approach is analyzed exhaustively in scientific community and used a lot in industry, it has a few drawbacks. First, it can not recommend songs to **new users**, whose preferences are unknown to the recommender system. (This is called the *new-user* problem in the literature) Second, it can not recommend **new songs** to users (Called the *new-song* problem) 

Another problem shared by many recommender systems is the lack of incorporating user feedback into the system. Typically, the aim of a recommender system is to choose the most likely songs that the user will like. That is, the system **exploits** the taste of the user. However, the system could have recommended new songs just to see how the user would react, and improve its belief on the preference of user. This could have a great impact on the accuracy of the predictions. This is called **exploration**.(!!! formulate better !!!) .This dilemma is explained in more detail in the next section.

### Reinforcement Learning / Multi-Armed Bandit

A reinforcement learning algorithm actively explores its environment to gain information and exploit the knowledge at hand to make predictions or decisions.

Multi-armed bandit is a well studied reinforcement learning problem where the aim is to allocate resources to alternative choices such that th cumulative reward from these choices are maximized. Usually the alternative choices are conceptualized as arms of a slot machine (hence, the name **multi-armed** bandit) and resources as pulling each arm of the machine. Specifically, suppose the slot machine has $M$ arms, $\mathcal{A} = \{1,2, \dots, M\}$ and probability distributions $\mathcal{D} = \{D_1, D_2, \dots, D_M\}$ corresponding to each arm. For each round $t$ in $\{1,2,\dots,n\}$, the player can choose an index, $I_t \in \mathcal{A}$ and in return gets a reward $r_{I_t} \sim D_{I_t}$. Finally, the aim is to maximize the cumulative reward
\begin{equation}
\sum_{t = 1}^{n} r_{I_t}
\end{equation}

(!!! explain exploration-exploitation !!!)

## Personalized User Rating Model

### Music Audio Content

A user's preference can be represented as:

$$U_c = \theta'x$$

where $\theta$ represents user preference of different music features and $x$ is the feature vector of audio content of a song. 

### Novelty

Novelty can be defined as repeating songs at proper frequencies. In this work, it's defined as: $ 1 - \frac{\texttt{number of unique songs}}{\texttt{listening history length}}$. Let $t$ be the time elapsed
since the last listening of the song, the novelty recovers following the function:

$$U_n = 1 - e^{-t/s}$$

where $s$ is a parameter indicating the recovery speed, with slower recovery having a higher s. 

### Combined Model

A user's preference of a recommendation can be represented as a rating; the
higher the rating is, the more the user likes the recommendation. Unlike traditional recommenders which assume ratings are static, we assume that a rating is the combined effect of the user’s preference of the song's content and the dynamically changing novelty.

$$U = U_cU_n = \theta'x(1 - e^{-t/s})$$

## Interactive Music Recommendation
![Interactive Music Recommendation](interactive-music-recommendation.png)

Under the rating model, each user is represented by a set of parameters $\Omega = \{\theta, s\}$. However, $\Omega$ needs to be estimated from historical data, and thus uncertainty always exists. In this case, the greedy strategy used by traditional systems is suboptimal, and it is necessary to take the uncertainty into account and balance exploration and exploitation. . As illustrated in above figure, we treat songs as arms, and user ratings as payoffs. The music recommendation problem is then transformed into a multi-armed bandit problem, and the objective of a music recommender is also changed to maximizing the sum of the ratings given by the target user *over the long term*. 

Bayes-UCB algorithm is adopted for recommendation task. First we denote the rating given by the target user to recommendation $i$ as a random variable
$R_i$, and the expectation of $R_i$ is $U_i$ given the feature vector $(x_i,t_i)$:
$$ \mathbb{E}[R_i] = U_i = \theta'x_i(1 - e^{-t_i/s})$$

The expected rating of song $k$, denoted as $U_k$ can be predicted as:

$$ \lambda_{k}^{l} = p(U_k|D_l) = \int p(U_k|\Omega)p(\Omega|D_l)d\Omega$$

where $D_l = \{(x_i,t_i,r_i)\}_{i=1}^{l}$ are the accumulated $l$ history 
recommendations.

To balance exploration and exploitation, Bayes-UCB recommends song $k^{*}$, which maximizes the quantile function:

$$k^{*} = arg \max_{k=1\ldots|S|} Q(\alpha, \lambda_{k}^{l})$$

where $Q$ satisfies $\mathbb{P}[U_k \leq Q(\alpha, \lambda_{k}^{l})] = \alpha$ and $S$ is all songs in database. 

### Pseudocode for BayesUCB

`for` $l = 1$ ```to``` $n$ ```do
    for all song``` $k = 1, \ldots, |S|$``` do
        compute``` $q_{k}^{l} = Q(1-1/l, \lambda_{k}^{l-1})$
    ```end for
    recommend song``` $k^{*} = arg \max_{k=1\ldots|S|} q_{k}^{l}$
    ```gather rating``` $r_l$; ```update``` $p(\Omega|D_l)$  ```and``` $\lambda_{k}^{l}$``    `
end for```



## Bayesian Models and Inference

### Exact Bayesian Model

$$R|x,t,\theta,s,\sigma^{2} \sim \mathcal{N}(\theta'x(1-e^{-t/s}), \sigma^{2})$$
$$\theta|\sigma^{2} \sim \mathcal{N}(0, a_0\sigma^2 I)$$
$$s \sim \mathcal{G}(b_0, c_0)$$
$$\tau = 1 / \sigma^{2} \sim \mathcal{G}(f_0, h_0)$$

### Approximate Bayesian Model
$$R|x,t,\theta, \beta, \sigma^{2} \sim \mathcal{N}(\theta'x\beta't, \sigma^{2})$$
$$\theta|\sigma^{2} \sim \mathcal{N}(\mu_{\theta_0}, \sigma^2 D_0)$$
$$\beta | \sigma^{2}  \sim \mathcal{G}(\mu_{\beta_0},  \sigma^2 E_0)$$
$$\tau = 1 / \sigma^{2} \sim \mathcal{G}(a_0, b_0)$$

where $\theta,\beta, \tau$ are parameters. $D_0,E_0,\mu_{\beta_0}, a_0, b_0$ are hyperparameters of the priors to be specified beforehand. $D_0$ and $E_0$ are positive definite matrices. 